In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Tim\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [3]:
# open dictionary to fill to put into MongoDB
scraped_data = {}

In [4]:
# NASA news scraping
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(1)

data = BeautifulSoup(browser.html)
slide = data.find("li", {"class": "slide"})
news_title = slide.find("div", {"class": "content_title"}).text.strip()
news_p = slide.find("div", {"class": "article_teaser_body"}).text.strip()

In [5]:
# featured URL
base = "https://www.jpl.nasa.gov"
url = f"{base}/spaceimages/?search=&category=Mars"
browser.visit(url)
time.sleep(1)

browser.find_by_id("full_image").click()
time.sleep(1)

browser.find_link_by_partial_text("more info").click()
time.sleep(1)

data = BeautifulSoup(browser.html)
image = data.find("img", {"class": "main_image"})

featured_image_url  = base + image["src"]

C:\Users\Tim\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [6]:
# Mars facts scraping
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)

dfs = pd.read_html(browser.html)
df = dfs[0]
df.columns = ["Statistic", "Value"]
mars_facts = df.to_html(index=False)

In [7]:
# Hemisphere data scraping

base = "https://astrogeology.usgs.gov"
url = f"{base}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

data = BeautifulSoup(browser.html)
links = data.find("div", {"class": "results"}).findAll("a", {"class": "itemLink"})

In [8]:
#filter out non image results
realLinks = []
for link in links:
    image = link.find("img")
    if (image):
        realLinks.append(base + link["href"]) # append the base url

# LOOP through each image link, click, grab the image info
hemisphere_data = []
for realLink in realLinks:
    browser.visit(realLink)
    time.sleep(1)
    
    data = BeautifulSoup(browser.html)
    hemi_url = data.find("ul").find("li").find("a")["href"]
    hemi_title = data.find("h2", {'class', "title"}).text.split(" Enhanced")[0]
    
    hemisphere_data.append({"title": hemi_title, "url": hemi_url})

In [9]:
#exit browser
browser.quit()

In [10]:
#add data to dictionary for mongo
scraped_data["news_title"] = news_title
scraped_data["news_p"] = news_p
scraped_data["featured_image_url"] = featured_image_url
scraped_data["mars_facts"] = mars_facts
scraped_data["hemispheres"] = hemisphere_data

In [11]:
# look at the scraped data

scraped_data

{'news_title': "Celebrate the Perseverance Rover Landing With NASA's Student Challenge",
 'news_p': 'The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16101_hires.jpg',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Statistic</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n